# Baseline Small Model

## Load Data

In [1]:
from datasets import load_dataset

dataset = load_dataset("billingsmoore/LotsawaHouse-bo-en", split='train').train_test_split(.1)
dataset['train'][0]

README.md:   0%|          | 0.00/886 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/107525 [00:00<?, ? examples/s]

{'bo': 'སེམས་ཅན་ཐམས་ཅད་ཉོན་མོངས་དུག་ལྔ་རགས།།',
 'en': 'The five poisons of the negative emotions grow more violent in all sentient beings—',
 'topic': 'Prayers, Guru Rinpoche Prayers, Guru Padmasambhava, Compendium of Sādhanas, Tibetan Masters, Tulku Zangpo Drakpa'}

In [2]:
dataset.save_to_disk('ds')

Saving the dataset (0/1 shards):   0%|          | 0/96772 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10753 [00:00<?, ? examples/s]

## Load Model

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained('billingsmoore/getok-v0')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor, EarlyStoppingCallback
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

## Preprocess Data

In [ ]:
def translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['bo']]
    translation_targets = [example for example in examples['en']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=256, truncation=True, padding="max_length")
    
    
    return translation_model_inputs

In [ ]:
tokenized_dataset = dataset.map(translation_preprocess_function, batched=True)

## Define Evaluation Metric

In [ ]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

## Train Model

In [ ]:
%env WANDB_PROJECT=efficient

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"baseline-small",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    processing_class=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()